In [ ]:
############ TUKIYE FILES
import geopandas as gpd

# Check the georeference stystem
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Turkiye/Mugla_Koycegiz/progression_koycegiz_29072021.shp')
print(gdf.crs)

# gdf = gdf.to_crs(epsg=32719)
# print(gdf.crs)

EPSG:32635
EPSG:32719


In [1]:
## KOYCEGIZ

import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Turkiye/Mugla_Koycegiz/progression_koycegiz_29072021.shp')
gdf = gdf.to_crs(epsg=5636)

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('zp_link', axis=1)
gdf = gdf.drop('burn_perio', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('source', axis=1)

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort attributes
gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('tukey_202107_koycegiz_v1.geojson', driver='GeoJSON')




In [2]:
## MARMARIS
import pandas as pd
import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Turkiye/Mugla_Marmaris/progression_marmaris_29072021.shp')
gdf = gdf.to_crs(epsg=5636)

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('zp_link', axis=1)
gdf = gdf.drop('burn_perio', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('turket_202107_marmaris_v1.geojson', driver='GeoJSON')


In [3]:
## MERKEZ
import pandas as pd
import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Turkiye/Mugla_Merkez/progression_mugla_merkez_02082021.shp')
gdf = gdf.to_crs(epsg=5636)

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('zp_link', axis=1)
gdf = gdf.drop('burn_perio', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('turkey_202108_merkez_v1.geojson', driver='GeoJSON')



In [4]:
## KARACAHISAR

import geopandas as gpd
gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Turkiye/Mugla_Milasa_Karacahisar/progression_karacahisar_31072021.shp')
gdf = gdf.to_crs(epsg=5636)

# Rename attributes
gdf = gdf.rename(columns={
    'date_hour': 'iso_timestamp'
})

# Delete attribute
gdf = gdf.drop('zp_link', axis=1)
gdf = gdf.drop('burn_perio', axis=1)
gdf = gdf.drop('type', axis=1)
gdf = gdf.drop('source', axis=1)
gdf = gdf.drop('Area_QGIS', axis=1)

## Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

# Sort ids with time stamps -> from early to later
gdf = gdf.sort_values('iso_timestamp')

# Prepare agg dict: sumar numéricos, 'first' para el resto
agg_dict = {}
for col in gdf.columns:
    if col in ['geometry', 'iso_timestamp']:
        continue
    if pd.api.types.is_numeric_dtype(gdf[col].dtype):
        agg_dict[col] = 'sum'
    else:
        agg_dict[col] = 'first'

# Convine geometries con la misma 'iso_timestamp'
gdf_2 = gdf.dissolve(by='iso_timestamp', aggfunc=agg_dict)

# devolver iso_timestamp a columna (dissolve lo pone como índice)
gdf_2 = gdf_2.reset_index()

# Recalcular área y perímetro sobre las geometrías combinadas (área de la UNIÓN)
gdf_2['area_ha'] = gdf_2.geometry.area / 10000
gdf_2['perimeter_m'] = gdf_2.geometry.length

# CReate a new 'id' secuancial (starting in 1)
gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

# Crear geometrías acumuladas, áreas y perímetros progresivos
from shapely.ops import unary_union

geom_union = None
acum_rows = []

for idx, row in gdf_2.iterrows():
    if geom_union is None:
        geom_union = row.geometry
    else:
        geom_union = unary_union([geom_union, row.geometry])
    # Disuelve por si quedan multipolígonos separados
    geom_union = unary_union(geom_union)
    area_acum = geom_union.area / 10000
    perim_acum = geom_union.length
    acum_rows.append({
        'id': idx + 1,
        'order_field': idx + 1,
        'iso_timestamp': row['iso_timestamp'],
        'geometry': geom_union,
        'area_ha': area_acum,
        'perimeter_m': perim_acum
    })

gdf_acum = gpd.GeoDataFrame(acum_rows, crs=gdf_2.crs)


# Convertir a WGS84 solo para exportar y compatibilidad con QGIS
gdf_acum = gdf_acum.to_crs(epsg=4326)
gdf_acum.to_file('turkey_202108_karacahisar_v1.geojson', driver='GeoJSON')
